# Prediction Project - Acceptance Rate (%) 

# Import Libraries

In [1]:
import psycopg2
import pandas as pd
import seaborn as sns
import numpy as np
import datetime
import xgboost as xgb
import pickle

from matplotlib import pyplot as plt
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import sys
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import os

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, accuracy_score, f1_score, fbeta_score, precision_score
from sklearn.metrics import mean_squared_error, r2_score, recall_score, average_precision_score, log_loss
# from sklearn.externals import joblib
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE

import optuna
from optuna.samplers import TPESampler
from optuna.integration import XGBoostPruningCallback

from flask import Flask, jsonify
from flask_cors import CORS

In [2]:
# conda update --all
# conda install -c conda-forge xgboost
# conda install graphviz python-graphviz

# Connect Redshift

In [3]:
def connect_redshift(credentials):  
    
    # connect redshift
    connection = psycopg2.connect(
        host=credentials['host'],
        port=credentials['port'],
        dbname=credentials['dbname'],
        user=credentials['user'],
        password=credentials['password'])
    
    # initialize cursor objects
    cur = connection.cursor()
    
    return cur   

# Import Data

In [4]:
def load_data(cur, query):    
    
    # load data
    cur.execute(query)
    
    # frame the data
    df = pd.DataFrame(cur.fetchall())
    df.columns = [desc[0] for desc in cur.description]
    
    return df

# Pre-Modeling

### 1. Data Processing

#### 1.0 Outliers Functions

In [5]:
# create a function which remove samples, where a given feature is above/below a given threshold.

def remove_outliers(df, feature:str, threshold:int, above_or_below:str):    
    
    # find index of outliers
    if above_or_below =='above':
        drop_index = set(df[df[feature]>threshold].index) 
    elif above_or_below =='below':
        drop_index = set(df[df[feature]<threshold].index)
    else:
        raise ValueError('Wrong above_or_below input')
    
    # get index of df without outliers
    new_index = list(set(df.index) - set(drop_index))
    
    # remove outliers
    df = df.iloc[new_index]
    
    # reset index
    df.reset_index(drop=True, inplace=True)
    
    return df

#### 1.1 Bucketing Functions

In [6]:
# get the top X most common values of a feature

def feature_top_x(df, feature, top_x=None):
    data = df[feature].value_counts(normalize=True).head(top_x).to_frame().reset_index()
    top_x_data = list(data.iloc[:,0])
    return top_x_data

In [7]:
# create X buckets of a feature

def feature_bucket_based_on_top_x(row, feature, top_x_list):      
    if pd.isnull(row[feature]):
        return None
    elif(row[feature] in top_x_list):
        return row[feature]
    else:
        return 'Other'

In [8]:
# apply the relevant bucket for each sample in the data set 

def calculate_feature_bucked(df, feature, top_x):
    top_x_list = feature_top_x(df, feature, top_x)
    df[f"{feature}_bucket"] = df.apply(lambda row: feature_bucket_based_on_top_x(row, feature, top_x_list) ,axis=1)
    return df

#### 1.2 Data Processing Function

In [9]:
def data_processing(df, outliers_to_be_removed):         
    
    # remove outliers
    for feature, values in outliers_to_be_removed.items():
        df = remove_outliers(df, feature, values['threshold'], values['above_or_below'])
    
    # create buckets of categorical features
    features_to_bucket = {'most_played_game':12,
                          'country':10,
                          'language':10
                         }
    # convert categorical features to buckets
    for k, v in features_to_bucket.items():
        df =  calculate_feature_bucked(df,k,v) 

    # drop columns
    col_to_drop = ['channel_id', 'application_id', 
                   'country', 'language', 'most_played_game']
    df = df.drop(col_to_drop, axis=1)

    # convert numeric features to category
    numeric_to_category = ['is_tipping_panel', 'is_bot_command_usage',
                           'is_overlay', 'is_website_visit',
                           'is_se_live', 'is_alert_box_fired',
                           'is_open_stream_report']
    for feature in numeric_to_category:
        df[feature] = df[feature].astype(object)
      
    return df

#### 1.3 Data Preparation Function

In [10]:
def data_split(df):    
    
    # Declare feature vector and target variable
    X = df.drop('is_accept', axis=1)
    y = df['is_accept']
    
    # Create dummy variables
    X = pd.get_dummies(X)
    
    # split the dataset
    return train_test_split(X, y, test_size=0.3, random_state = 42, stratify=y)

### 2. Models Evaluation

#### 2.1 Evaluation Functions

In [11]:
def model_performance(model, X_test, y_test):
    res = {}
    
    # predict with the model
    y_pred = model.predict(X_test)

    # model scoring
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)
    logistic_loss = log_loss(y_test, y_pred)
    
    # print scores
    res = {'RMSE': rmse, 'R2': r2, 'Log Loss': logistic_loss}
    
    return res

# Modeling

### 1. Hyperparameters Optimization

#### 1.1 Optuna Optimization Function

In [12]:
# disable printing
optuna.logging.set_verbosity(optuna.logging.WARNING)

# create objective function which returns scoring (numerical value to evaluate the performance of the hyperparameters)
def objective(trial, eval_model, param, score_func, X_train, y_train, X_test, y_test):    
    
    # hyperparameters to be tuned
    hyperparameters_candidates = {
        "max_depth": trial.suggest_int("max_depth", 4, 12),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.005, 0.05),
        "colsample_bytree": trial.suggest_loguniform("colsample_bytree", 0.2, 0.6),
        "subsample": trial.suggest_loguniform("subsample", 0.4, 0.8),
        "alpha": trial.suggest_loguniform("alpha", 0.01, 10.0),
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 10.0),
        "gamma": trial.suggest_loguniform("lambda", 1e-8, 10.0),
        "min_child_weight": trial.suggest_loguniform("min_child_weight", 10, 1000),
        "scale_pos_weight": trial.suggest_int('scale_pos_weight', 1, 100)
    }
    
    # instantiate the model 
    xgb_optuna = eval_model(**param, **hyperparameters_candidates)
    
    # Add a callback for pruning (ensure unpromising trials are stopped early)
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation_0-aucpr")
    
    # fit
    xgb_optuna.fit(X_train,
                   y_train,
                   verbose=False,
                   eval_set=[(X_test, y_test)],
                   callbacks=[pruning_callback]) 
    
    # tag each trial with keyword
    trial.set_user_attr(key="model", value=xgb_optuna)
    
    # predict with the model
    y_pred = xgb_optuna.predict(X_test)


    # score the model
    score = score_func(y_pred)

    return score

#### 1.2 Retrieving Function

In [13]:
# create a callback function to retrive the best model (i.e. the model with the best hyperparameters)
def callback(study, trial):
    if study.best_trial.number == trial.number:
        study.set_user_attr(key="best_model", value=trial.user_attrs["model"])

### 2. Loss Functions 

In [14]:
class AveragePrecisionScore:
    def __init__(self, y_test, direction):
        self.y_test = y_test
        self.direction = direction
        self.name = "average_precision_score"
    def score(self, y_pred):
        return average_precision_score(self.y_test, y_pred)

    
class LogLoss:
    def __init__(self, y_test, direction):
        self.y_test = y_test
        self.direction = direction
        self.name = "log_loss"
    def score(self, y_pred):
        return log_loss(self.y_test, y_pred)


    
class RMSE:
    def __init__(self, y_test, squared, direction):
        self.y_test = y_test
        self.squared = False
        self.direction = direction
        self.name = "mean_squared_error"
    def score(self, y_pred):
        return mean_squared_error(self.y_test, y_pred, squared=self.squared)

### 3. Execution

#### 3.1 Execution Functions

In [15]:
def tune_models_hyperparams(eval_model, param, X_train, y_train, X_test, y_test):
    
    # dictionary for saving models
    grids = {}


    # define scoring method
    scoring_objects = [AveragePrecisionScore(y_test, direction="maximize"),
                       LogLoss(y_test, direction="minimize"),
                       RMSE(y_test, squared=False, direction="minimize")
                      ]


    # create a sampler object to find more efficiently the best hyperparameters
    sampler = TPESampler()  # by default the sampler = TPESampler()


    for score_obj in scoring_objects:

        # create a study object to set the direction of optimization and the sampler
        study = optuna.create_study(sampler=sampler, direction = score_obj.direction)

        # run the study object
        study.optimize(lambda trial: objective(trial,
                                               eval_model,
                                               param,
                                               score_obj.score,
                                               X_train,
                                               y_train,
                                               X_test,
                                               y_test), # make smart guesses where the best values hyperparameters
                       n_trials = 1, # try hyperparameters combinations n_trials times
                       callbacks=[callback]) # callback save the best model

        # name the best model
        model_name = "xgb_reg_optuna_" + score_obj.name
        grids[model_name] = {}

        # initiate best model
        model_object = study.user_attrs["best_model"]
        grids[model_name]["model_object"] = model_object

        # score best model
        model_scores = model_performance(model_object, X_test, y_test)
        grids[model_name]["model_scores"] = model_scores
    
    return grids

#     # open a file where we store the model
#     model_file = open(f'./{model_name}.pkl', 'wb')
#     grids[model_name]["model_file"] = model_file
                      
#     # dump model to that  file
#     pickle.dump(model_object, model_file) 
    
#     # close the file
#     model_file.close()

In [16]:
def print_grid_results(grids):   
    
    # print models results
    for name, model in grids.items():
        print('{:-^70}'.format(' [' + name + '] '))
        print(model["model_scores"])

In [17]:
def get_best_model(grids):     
    
    # filter models with negative R^2 (models worse than a constant function that always predicts the mean of the data)
    new_grids = {k: v for k, v in grids.items() if v["model_scores"]['R2']>0}
    
    # retrieve best model
    if bool(new_grids):
        best_model_item = min(new_grids.items(), key=lambda x: x[1]["model_scores"]['Log Loss'])
        best_model_object = best_model_item[1]["model_object"]
    else: 
        raise ValueError("\033[1m" + 'No model has fitted the data well.' + "\033[0m")
            
    return best_model_object

In [18]:
def save_best_model(best_model_object):    
    
    # open (and close) a file where we store the best model
    with open('model.pkl', 'wb') as f:
        
        # dump best model to the file 
        pickle.dump(best_model_object, f)

#### 3.2 Parameters

In [19]:
credentials = {'host':"172.31.0.207",
               'port':5439,
               'dbname':"streamelements",
               'user':"yehuda",
               'password':'SEdontdaytrade1337ageofempires'}

In [20]:
query = f"""
select channel_id
     , application_id
     , se_age_days::float
     , country
     , language
     , ccv_30_d::float
     , ccv_60_d::float
     , ccv_growth_60_30_d::float
     , most_played_game
     , cnt_streams::float
     , invitations_l3m::float
     , acceptances_l3m::float
     , deployments_l3m::float
     , rejections_l3m::float
     , offer_page_visits_l3m::float
     , invitations_l6m::float
     , acceptances_l6m::float
     , deployments_l6m::float
     , rejections_l6m::float
     , offer_page_visits_l6m::float
     , hours_streamed::float
     , hours_watched::float
     , total_chatters::float
     , is_tipping_panel
     , is_bot_command_usage
     , is_overlay
     , is_website_visit
     , is_se_live
     , is_alert_box_fired
     , cnt_alert_box_fired::float
     , is_open_stream_report
     , campaigns_revenue::float
     , tips::float
     , tips_revenue::float
     , on_screen_cheers::float
     , on_screen_cheers_revenue::float
     , on_screen_subs::float
     , on_screen_subs_revenue::float
     , is_accept
     , campaign_game
--     , campaign_max_payout::float
--     , campaign_max_payout_per_ccv::float
from bi_dev.acceptance_rate_model_training
where campaign_category = 'games' and
      se_age_days>0 and
      invite_timestamp >= '2022-01-01' 
"""

In [21]:
# outliers to be removed
outliers_to_be_removed = {'ccv_30_d':{'threshold':700,'above_or_below':'above'}}

In [22]:
# model parameters
param = {"objective": "binary:logistic", # logistic regression for binary classification, output probability
         "missing": np.nan, # whenever a null values is encountered it is treated as missing value
         "seed": 42, # used to generate the folds
         "tree_method": "gpu_hist", # speed up processing by using gpu power
         "early_stopping_rounds": 50, # overfitting prevention, stop early if no improvement in learning
         "eval_metric": "aucpr", # evaluation metric for validation data
         "n_estimators": 10000 # number of trees
        }

### 4. Run

In [23]:
def main(credentials, query, param, outliers_to_be_removed):     
    
    # connect redshift
    cur = connect_redshift(credentials)   
    
    # load data from redshift
    df = load_data(cur, query)
        
    # data processing 
    df = data_processing(df, outliers_to_be_removed) 
    
    # data split
    X_train, X_test, y_train, y_test = data_split(df) 
    
    # define model to pass
    eval_model = xgb.XGBRegressor 
    
    # save models in a dictionary 
    grids = tune_models_hyperparams(eval_model, param, X_train, y_train, X_test, y_test) 
    
    # print models
    print_grid_results(grids) 
    
    # extract best model
    best_model_object = get_best_model(grids) 
    
    # save best model in a pickle file
    save_best_model(best_model_object) 
    
    return best_model_object

In [27]:
%%time
main(credentials, query, param, outliers_to_be_removed)

In [26]:
if __name__ == "__main__":
    model_training.run(debug=True)